## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**618. Student Report By Geography (Hard)**

**Table: Student**

| Column Name | Type    |
|-------------|---------|
| name        | varchar |
| continent   | varchar |

This table may contain duplicate rows.
Each row of this table indicates the name of a student and the continent they came from.
 
A school has students from Asia, Europe, and America.

**Write a solution to pivot the continent column in the Student table so that each name is sorted alphabetically and displayed underneath its corresponding continent. The output headers should be America, Asia, and Europe, respectively.**

The test cases are generated so that the student number from America is not less than either Asia or Europe.

The result format is in the following example.

**Example 1:**

**Input:** 

**Student table:**

| name   | continent |
|--------|-----------|
| Jane   | America   |
| Pascal | Europe    |
| Xi     | Asia      |
| Jack   | America   |

**Output:** 
| America | Asia | Europe |
|---------|------|--------|
| Jack    | Xi   | Pascal |
| Jane    | null | null   |
 

**Follow up:** If it is unknown which continent has the most students, could you write a solution to generate the student report?

In [0]:
student_data_618 = [
    ("Jane", "America"),
    ("Pascal", "Europe"),
    ("Xi", "Asia"),
    ("Jack", "America"),
]

student_columns_618 = ["name", "continent"]
student_df_618 = spark.createDataFrame(student_data_618, student_columns_618)
student_df_618.show()

+------+---------+
|  name|continent|
+------+---------+
|  Jane|  America|
|Pascal|   Europe|
|    Xi|     Asia|
|  Jack|  America|
+------+---------+



In [0]:
student_df_618 = student_df_618.dropDuplicates()

In [0]:
window_spec = Window.partitionBy("continent").orderBy("name")

student_ranked_df_618 = student_df_618.withColumn(
    "rn", 
    row_number().over(window_spec)
)

In [0]:
america_df_618 = student_ranked_df_618.filter(col("continent") == "America") \
    .select(col("rn"), col("name").alias("America"))

asia_df_618 = student_ranked_df_618.filter(col("continent") == "Asia") \
    .select(col("rn"), col("name").alias("Asia"))

europe_df_618 = student_ranked_df_618.filter(col("continent") == "Europe") \
    .select(col("rn"), col("name").alias("Europe"))


In [0]:
america_df_618 \
    .join(asia_df_618, on="rn", how="left") \
    .join(europe_df_618, on="rn", how="left").drop('rn').show()


+-------+----+------+
|America|Asia|Europe|
+-------+----+------+
|   Jack|  Xi|Pascal|
|   Jane|NULL|  NULL|
+-------+----+------+

